In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pwd

'/home/bene/projects/generative-models/examples/torch_autoencoder'

In [3]:
%cd ../..
%pwd

/home/bene/projects/generative-models


'/home/bene/projects/generative-models'

In [4]:
import torch
from gempy.torch.encoder import ConvEncoder
from gempy.torch.decoder import ConvDecoder
from gempy.torch.auto_encoder import AutoEncoder

ModuleNotFoundError: No module named 'torch'

In [ ]:
input_dim = (1, 28, 28)
z_dim = (4, )
z_labels = ('z', )

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    print (f'GPU available')
    device = 'cuda'

x_random = torch.randn(1, *input_dim)
x_random = x_random.to(device)

In [ ]:
encoder = ConvEncoder(
    input_dim=input_dim,
    filters=(32, 64, 64, 64),
    #filters=(8, 16, 16, 16),
    kernel_size=(3, 3, 3, 3),
    strides=(1, 2, 2, 1),
    activation='leaky_relu',
    latent_dim=z_dim,
    latent_labels=z_labels,
    latent_activation='sigmoid',
)

encoer = encoder.to(device)
encoder

In [ ]:
decoder = ConvDecoder(
    latent_dim=z_dim,
    latent_upscale=(64, 3, 3),
    filters=[64, 64, 32, 1],
    #filters=[16, 16, 8, 1],
    kernel_size=[3, 4, 4, 3],
    strides=[1, 2, 2, 1],
    activation=['leaky_relu', 'leaky_relu', 'leaky_relu', 'sigmoid'],
    latent_merge=False,
    latent_activation=None,
)

decoder = decoder.to('cuda')
decoder

In [ ]:
for l in decoder.conv_stack:
    print('layer activation: ', l[-2])

In [ ]:
auto_encoder = AutoEncoder(
    encoder=encoder,
    decoder=decoder,
)

auto_encoder = auto_encoder.to(device)
auto_encoder

In [ ]:
print('input shape     :', auto_encoder.encoder.conv_stack_shape_in)
print('latent shape     :', auto_encoder.encoder.conv_stack_shape_out)
print('z shape          :', z_dim)
print('output shape    :', auto_encoder.decoder.conv_stack_shape_out)

In [ ]:
y = auto_encoder(x_random)

print('latent space    :', encoder.latent_torch)
print('output shape    :', y.shape)


In [ ]:
for p in auto_encoder.parameters():
    print(p.shape)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

def get_cifar10(batch_size, dataset_directory, dataloader_workers):
    # Prepare dataset for training
    train_transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    train_dataset = torchvision.datasets.CIFAR10(root=dataset_directory, train=True,
                                                 download=True, transform=train_transformation)
    
    test_dataset =  torchvision.datasets.CIFAR10(root=dataset_directory, train=False,
                                                 download=True, transform=train_transformation)

    # Prepare Data Loaders for training and validation
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                               pin_memory=True, num_workers=dataloader_workers)

    return train_dataset, train_loader

def get_MNIST(batch_size, dataset_directory, dataloader_workers):
    # Prepare dataset for training
    train_transformation = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])
    
    train_dataset = torchvision.datasets.MNIST(root=dataset_directory, train=True,
                                               download=True, transform=train_transformation)
    
    test_dataset =  torchvision.datasets.MNIST(root=dataset_directory, train=False,
                                               download=True, transform=train_transformation)

    # Prepare Data Loaders for training and validation
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                               pin_memory=True, num_workers=dataloader_workers)

    return train_dataset, train_loader

In [ ]:
batch_size = 32
lr = 5e-4

train_dataset, train_loader = get_MNIST(batch_size, 'examples/torch_autoencoder/data/MNIST', 12)

In [ ]:
# create an optimizer object
optimizer = torch.optim.Adam(auto_encoder.parameters(), lr=lr)

In [ ]:
# mean-squared error loss
criterion = torch.nn.MSELoss()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(train_dataset[10][0].detach().numpy().transpose(1,2,0))

In [ ]:
epochs = 100

for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        # batch_features = batch_features.view(-1, 784).to(device)
        batch_features = batch_features.to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = auto_encoder(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

In [ ]:
import numpy as np

z_rand = np.random.rand(*z_dim)
t_rand = torch.from_numpy(z_rand[None, ...]).float().to(device)

y_rand = auto_encoder.decoder(t_rand)

y = y_rand[0].detach().cpu().numpy()
plt.imshow(y.transpose(1,2,0))